In [1]:
#imports
import pandas as pd
import numpy as np
import datetime

from sqlalchemy import create_engine, inspect, text

In [2]:
#load data
wildfires = pd.read_csv("California_Fire_Incidents.csv")
wildfires.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [3]:
#check dtypes
wildfires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      105 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [4]:
#clean up datetime format
wildfires['Started'] = pd.to_datetime(wildfires['Started'], format='mixed')
wildfires['Updated'] = pd.to_datetime(wildfires['Updated'], format='mixed')
wildfires['Extinguished'] = pd.to_datetime(wildfires['Extinguished'], format='mixed')

#create 2 new columns: 1) year wildfire started; 2) month wildfire started
wildfires['YearStarted'] = wildfires['Started'].dt.year
wildfires['MonthStarted'] = wildfires['Started'].dt.month
#get rid of 2 1969 entires
wildfires = wildfires[wildfires.YearStarted != 1969]

In [5]:
#check out new column names
wildfires.head(5)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders,YearStarted,MonthStarted
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06 18:30:00+00:00,NaN,2013,8
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08 18:30:00+00:00,NaN,2013,5
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30 18:00:00+00:00,NaN,2013,7
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30 08:00:00+00:00,NaN,2013,8
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11 06:30:00+00:00,11.0,2013,5


In [6]:
sorted_df = wildfires.sort_values(by='YearStarted', ascending=True)
sorted_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders,YearStarted,MonthStarted
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06 18:30:00+00:00,NaN,2013,8
104,75.0,False,CAL FIRE Madera-Mariposa-Merced Unit,NaN,2013,True,/incidents/2013/6/12/revis-fire/,NaN,NaN,Madera,...,Finalized,NaN,NaN,NaN,NaN,7055a440-5d4c-4f58-bac9-2954ff6698a5,2013-06-14 16:30:00+00:00,13.0,2013,6
105,75.0,False,"USFS San Bernardino National Forest, CAL FIRE ...",NaN,2013,True,/incidents/2013/5/13/lytle-fire/,"Full containment at 6:00 PM on May 15, 2013.",NaN,San Bernardino,...,Finalized,NaN,1.0,NaN,NaN,e2c4b308-9a2d-483b-8434-999bef9adc7c,2013-05-15 08:15:00+00:00,13.0,2013,5
106,75.0,False,CAL FIRE Sonoma-Lake-Napa Unit,NaN,2013,True,/incidents/2013/4/30/silverado-fire/,NaN,NaN,Napa,...,Finalized,NaN,NaN,NaN,NaN,d8d1c943-7b5d-4885-9043-f41075d6eded,2013-05-01 17:15:00+00:00,NaN,2013,4
107,73.0,False,CAL FIRE San Bernardino Unit,NaN,2013,True,/incidents/2013/9/11/liveoak-fire/,NaN,NaN,San Bernardino,...,Finalized,NaN,NaN,NaN,NaN,31f21c59-8a94-4b17-91aa-88ca14817162,2013-09-12 08:45:00+00:00,NaN,2013,9


In [7]:
wildfires.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1634 entries, 0 to 1635
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   AcresBurned           1631 non-null   float64            
 1   Active                1634 non-null   bool               
 2   AdminUnit             1634 non-null   object             
 3   AirTankers            28 non-null     float64            
 4   ArchiveYear           1634 non-null   int64              
 5   CalFireIncident       1634 non-null   bool               
 6   CanonicalUrl          1634 non-null   object             
 7   ConditionStatement    284 non-null    object             
 8   ControlStatement      105 non-null    object             
 9   Counties              1634 non-null   object             
 10  CountyIds             1634 non-null   object             
 11  CrewsInvolved         171 non-null    float64            
 12  Dozers     

In [8]:
#write to database
engine = create_engine("sqlite:///cali_wildfires.sqlite")

In [9]:
wildfires.to_sql(name="cali_wildfires", con=engine, index=False, if_exists="append", method="multi")

1634

In [10]:
#LOOK! it works!
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within the 'dow' table and its types
for table in tables:
    print(table)
    print("--------")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

cali_wildfires
--------
AcresBurned FLOAT
Active BOOLEAN
AdminUnit TEXT
AirTankers FLOAT
ArchiveYear BIGINT
CalFireIncident BOOLEAN
CanonicalUrl TEXT
ConditionStatement TEXT
ControlStatement TEXT
Counties TEXT
CountyIds TEXT
CrewsInvolved FLOAT
Dozers FLOAT
Engines FLOAT
Extinguished TIMESTAMP
Fatalities FLOAT
Featured BOOLEAN
Final BOOLEAN
FuelType TEXT
Helicopters FLOAT
Injuries FLOAT
Latitude FLOAT
Location TEXT
Longitude FLOAT
MajorIncident BOOLEAN
Name TEXT
PercentContained FLOAT
PersonnelInvolved FLOAT
Public BOOLEAN
SearchDescription TEXT
SearchKeywords TEXT
Started TIMESTAMP
Status TEXT
StructuresDamaged FLOAT
StructuresDestroyed FLOAT
StructuresEvacuated FLOAT
StructuresThreatened FLOAT
UniqueId TEXT
Updated TIMESTAMP
WaterTenders FLOAT
YearStarted INTEGER
MonthStarted INTEGER



In [11]:
#lets get outta here
engine.dispose()